<a href="https://colab.research.google.com/github/graziaperna/NarrativeIntelligence/blob/mt5/Fine_Tuning_IT5_MT5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
!pip install pipeline

In [36]:
!pip install transformers[torch]

In [37]:
!git clone https://github.com/gsarti/it5.git

fatal: destination path 'it5' already exists and is not an empty directory.


In [38]:
!pip install -r it5/requirements.txt

In [39]:
!shuf datasetT5.jsonl > datasetT5_shuf.jsonl
!cat datasetT5_shuf.jsonl | wc -l

7150


In [40]:
!head -n 6500 datasetT5_shuf.jsonl > train.json
!tail -n 650 datasetT5_shuf.jsonl > temp.json
!head -n 500 temp.json > test.json
!tail -n 150 temp.json > val.json
!rm temp.json
!cat train.json | wc -l
!cat test.json | wc -l
!cat val.json | wc -l

6500
500
150


In [41]:
!python it5/finetuning/run_seq2seq.py

2024-06-06 22:04:02.772440: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-06 22:04:02.772518: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-06 22:04:02.776864: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-06 22:04:06.753642: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
usage: run_seq2seq.py [-h] --model_name_or_path MODEL_NAME_OR_PATH [--config_name CONFIG_NAME]
                      [--tokenizer_name TOKENIZER_NAME] [--cache_dir CACHE_DIR]
                      [--use_fast_tokenizer [USE_FAST_TOKENIZER]] 

In [ ]:
#IT5 = gsarti/it5-small
#MT5 = google/mt5-small

!python it5/finetuning/run_seq2seq.py --model_name_or_path google/mt5-small --tokenizer_name google/mt5-small --do_train true --do_eval true  --do_predict true --num_train_epochs 5 --train_file train.json --validation_file val.json --test_file test.json --source_column source --target_column target --max_source_length 512 --max_target_length 256  --output_dir model_base_512_256_5/ --overwrite_output_dir true --per_device_train_batch_size 4 --per_device_eval_batch_size 4 --save_strategy epoch

In [43]:
!mkdir model_small
!cp model_small_512_256_5/*.* model_small/

mkdir: cannot create directory ‘model_small’: File exists
cp: cannot stat 'model_small_512_256_5/*.*': No such file or directory


In [44]:
!mkdir model_base
!cp model_base_512_256_5/*.* model_base/

mkdir: cannot create directory ‘model_base’: File exists


In [45]:
!tar -zcf model_small.tar.gz model_small/

In [46]:
!tar -zcf model_base.tar.gz model_base/

In [47]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [48]:
!cp model_small.tar.gz '/content/drive/My Drive/temp/'


cp: cannot create regular file '/content/drive/My Drive/temp/': Not a directory


In [49]:
!cp model_base.tar.gz '/content/drive/My Drive/temp/'


cp: cannot create regular file '/content/drive/My Drive/temp/': Not a directory


In [50]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

In [51]:
tokenizer = AutoTokenizer.from_pretrained("/content/model_small")
model = AutoModelForSeq2SeqLM.from_pretrained("/content/model_small")

ner_pipeline = pipeline(task="text2text-generation",model=model,tokenizer=tokenizer , max_new_tokens=256)

text = "La giornata spuntò serena e limpida per gli sposi che dopo aver riposato una notte a Como continuarono il loro viaggio verso la Tremezzina"
ner_results = ner_pipeline(text)
print(ner_results)

OSError: /content/model_small does not appear to have a file named config.json. Checkout 'https://huggingface.co//content/model_small/tree/None' for available files.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("/content/model_base_512_256_5")
model = AutoModelForSeq2SeqLM.from_pretrained("/content/model_base_512_256_5")

ner_pipeline = pipeline(task="text2text-generation",model=model,tokenizer=tokenizer, max_new_tokens=256 )

text = "La giornata spuntò serena e limpida per gli sposi che dopo aver riposato una notte a Como continuarono il loro viaggio verso la Tremezzina"
ner_results = ner_pipeline(text)
print(ner_results)

In [ ]:
# Specifica il percorso del file che vuoi salvare su Google Drive
file_path = '/content/model_base_512_256_5/'
drive_path = '/content/drive/MyDrive'

# Copia il file su Google Drive
!cp -r {file_path} {drive_path}

In [ ]:
# Specifica il percorso del file che vuoi salvare su Google Drive
file_path = '/content/model_base.tar.gz/'
drive_path = '/content/drive/MyDrive'

# Copia il file su Google Drive
!cp {file_path} {drive_path}

In [ ]:
# Specifica il percorso del file che vuoi salvare su Google Drive
file_path = '/content/model_small.tar.gz/'
drive_path = '/content/drive/MyDrive'

# Copia il file su Google Drive
!cp {file_path} {drive_path}